In [8]:
import concurrent.futures
from pathlib import Path

from virtual_lab.agent import Agent
from virtual_lab.constants import CONSISTENT_TEMPERATURE, CREATIVE_TEMPERATURE
from virtual_lab.prompts import create_merge_prompt
from virtual_lab.run_meeting import run_meeting

from nanobody_constants import (
    background_prompt,
    discussions_dir,
    model,
    num_iterations,
    num_rounds,
    principal_investigator,
    scientific_critic,
    immunologist,
    machine_learning_specialist,
    computational_biologist,
)

In [2]:
# Setup up topic to model mapping
topic_to_model = {
    "AlphaFold-Multimer": "ft:gpt-4o-mini-2024-07-18:personal:alphafold-multimer:ArGNLO1a",
    "ESM": "ft:gpt-4o-mini-2024-07-18:personal:esm:ArFrZ4tt",
    "Rosetta": "ft:gpt-4o-mini-2024-07-18:personal:rosetta:ArFx2nis",
    "SARS-CoV-2 spike protein": "ft:gpt-4o-mini-2024-07-18:personal:sars-cov-2-spike-protein:ArG9fZwz",
    "SARS-CoV-2 variants KP.3 and JN.1": "ft:gpt-4o-mini-2024-07-18:personal:sars-cov-2-variants-kp-3-and-jn-1:ArGVyHfW",
    "nanobodies": "ft:gpt-4o-mini-2024-07-18:personal:nanobodies:ArGjRZ9R",
}

In [3]:
# Setup up topic to agent mapping
topic_to_agent = {
    "AlphaFold-Multimer": computational_biologist,
    "ESM": machine_learning_specialist,
    "Rosetta": computational_biologist,
    "SARS-CoV-2 spike protein": immunologist,
    "SARS-CoV-2 variants KP.3 and JN.1": immunologist,
    "nanobodies": immunologist,
}

In [4]:
# Create specialized agents for each topic
specialized_agents = {}
for topic, base_agent in topic_to_agent.items():
    specialized_agent = Agent(
        title=f"{topic} {base_agent.title}",
        expertise=base_agent.expertise,
        goal=base_agent.goal,
        role=base_agent.role,
        model=topic_to_model[topic],
    )
    specialized_agents[topic] = specialized_agent

In [5]:
ablations_dir = discussions_dir / "ablations"
ablations_dir.mkdir(parents=True, exist_ok=True)

In [6]:
# Run meetings with base and specialized agents
agenda = "What are some of the mutations of the KP.3 and JN.1 variants of the SARS-CoV-2 spike protein and how might they affect nanobody binding?"

agents = [
    (immunologist, "base_immunologist"),
    (specialized_agents["SARS-CoV-2 variants KP.3 and JN.1"], "specialized_immunologist")
]

for agent, save_name in agents:
    run_meeting(
        meeting_type="individual", 
        team_member=agent,
        agenda=agenda,
        save_dir=ablations_dir,
        save_name=save_name,
        temperature=CONSISTENT_TEMPERATURE,
    )

Rounds (+ Final Round): 100%|██████████| 1/1 [01:17<00:00, 77.41s/it]


Input token count: 73
Output token count: 561
Tool token count: 0
Max token length: 634
Cost: $0.01
Time: 1:19


Rounds (+ Final Round): 100%|██████████| 1/1 [00:23<00:00, 23.21s/it]


Input token count: 102
Output token count: 565
Tool token count: 0
Max token length: 667
Time: 0:28


In [11]:
agenda = f"{background_prompt} Please create an antibody/nanobody design approach to solve this problem. Decide whether you will design antibodies or nanobodies. For your choice, decide whether you will design the antibodies/nanobodies de novo or whether you will modify existing antibodies/nanobodies. If modifying existing antibodies/nanobodies, please specify which antibodies/nanobodies to start with as good candidates for targeting the newest variant of the SARS-CoV-2 spike protein. If designing antibodies/nanobodies de novo, please describe how you will propose antibody/nanobody candidates."

agenda_questions = (
    "Will you design standard antibodies or nanobodies?",
    "Will you design antibodies/nanobodies de novo or will you modify existing antibodies/nanobodies (choose only one)?",
    "If modifying existing antibodies/nanobodies, which precise antibodies/nanobodies will you modify (please list 3-4)?",
    "If designing antibodies/nanobodies de novo, how exactly will you propose antibody/nanobody candidates?",
)

generic_lead = Agent(
    title="Lead Assistant",
    expertise="helping people with their problems",
    goal="help people with their problems",
    role="help people with their problems",
    model=model,
)

generic_team = [Agent(
    title=f"Assistant {i + 1}",
    expertise="helping people with their problems",
    goal="help people with their problems",
    role="help people with their problems",
    model=model,
) for i in range(4)]

teams = [
    (principal_investigator, [immunologist, machine_learning_specialist, computational_biologist, scientific_critic], "base_team"),
    (generic_lead, generic_team, "generic_team")
]

for team_lead, team_members, save_name in teams:
    run_meeting(
        meeting_type="team", 
        team_lead=team_lead,
        team_members=team_members,
        agenda=agenda,
        agenda_questions=agenda_questions,
        save_dir=ablations_dir,
        save_name=save_name,
        temperature=CONSISTENT_TEMPERATURE,
        num_rounds=num_rounds,
    )

Rounds (+ Final Round): 100%|██████████| 4/4 [05:18<00:00, 79.71s/it]


Input token count: 51,118
Output token count: 5,028
Tool token count: 0
Max token length: 6,854
Cost: $0.18
Time: 5:21
